In [ ]:
%pip install -q llama-index llama-index-vector-stores-chroma chromadb

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Establecemos los modelos que vamos a usar
EMBD_MODEL="text-embedding-3-small"
LLM_MODEL="gpt-4o-mini"

#### Indexing

##### 1.1 Load Documents from File

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

documents = SimpleDirectoryReader("resources").load_data()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

vector_store = ChromaVectorStore(chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents=documents,
    storage_context=storage_context
)

# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("What is the document about?")
response

##### 1.2 Load Document from Website

In [2]:
%pip install -q llama-index-readers-web

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from llama_index.readers.web import SimpleWebPageReader

urls = ["https://lilianweng.github.io/posts/2023-06-23-agent/"]
documents = SimpleWebPageReader().load_data(urls)

##### 2. Split Documents

In [31]:
from llama_index.core.text_splitter import TokenTextSplitter
from llama_index.core import Document

text_splitter = TokenTextSplitter(chunk_size=1024, chunk_overlap=200)
    
split_documents: list[str] = []
for doc in documents:
    chunks = text_splitter.split_text(doc.text)
    split_documents.extend(chunks)

##### 3. Embeddings and Store

In [32]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(
    model=EMBD_MODEL,
)

#### Retrieve and Generate

In [42]:
from llama_index.core import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(
    documents=[Document(text=text) for text in split_documents],
)
question = 'What are the approaches to Task Decomposition?'
retriever = index.as_retriever()

# Returns Node with score
nodes = retriever.retrieve(question)
nodes

[NodeWithScore(node=TextNode(id_='b8920c20-5383-4691-8981-711fbe1baf88', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e4a2087-327e-4c12-8bc9-122bf48a8dcb', node_type='4', metadata={}, hash='c68a1eaade2e8496c2a5aea772e5e2bfa7986a1d9dc4cb3f636098e9a615d938')}, metadata_template='{key}: {value}', metadata_separator='\n', text='<a href="#challenges" aria-label="Challenges">Challenges</a></li>\n                <li>\n                    <a href="#citation" aria-label="Citation">Citation</a></li>\n                <li>\n                    <a href="#references" aria-label="References">References</a>\n                </li>\n            </ul>\n        </div>\n    </details>\n</div>\n\n  <div class="post-content"><p>Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as <a href="https://github.com/Signi

In [46]:
from llama_index.llms.openai import OpenAI

QUERY_PROMPT = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {input} 
Context: {context} 
Answer:
"""

Settings.llm = OpenAI(model=LLM_MODEL)

query_engine = index.as_chat_engine(prompt=QUERY_PROMPT)

In [48]:
# question = 'En que consiste el enfoque Task Decomposition?'
# question = 'Quién es el mejor jugador de futbol segun la web?'

answer = query_engine.query(question)
answer.response

'The approaches to Task Decomposition include:\n\n1. **Chain of Thought (CoT)**: This technique enhances model performance by instructing the model to "think step by step." It breaks down complex tasks into smaller, manageable steps, allowing for clearer reasoning and problem-solving.\n\n2. **Tree of Thoughts**: This approach extends the Chain of Thought concept by exploring multiple reasoning possibilities at each step. It allows for a more comprehensive decomposition of tasks by considering various paths and outcomes, facilitating a deeper understanding of the task at hand.'

##### Advanced query

In [45]:
from llama_index.core import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor

# similarity postprocessor: filter nodes below 0.75 similarity score
processor = SimilarityPostprocessor(similarity_cutoff=0.75)
filtered_nodes = processor.postprocess_nodes(nodes)

response_syntetizer = get_response_synthesizer()

query_engine = index.as_query_engine(
    response_syntetizer=response_syntetizer
)
query_engine.query(question)

Response(response='Task Decomposition can be approached using techniques such as Chain of Thought (CoT) and Tree of Thoughts. CoT involves instructing the model to "think step by step," allowing it to break down complex tasks into smaller, manageable steps. This method enhances the model\'s performance on difficult tasks by transforming them into simpler components. Tree of Thoughts extends this concept by exploring multiple reasoning possibilities at each step, further aiding in the decomposition of tasks.', source_nodes=[NodeWithScore(node=TextNode(id_='b8920c20-5383-4691-8981-711fbe1baf88', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e4a2087-327e-4c12-8bc9-122bf48a8dcb', node_type='4', metadata={}, hash='c68a1eaade2e8496c2a5aea772e5e2bfa7986a1d9dc4cb3f636098e9a615d938')}, metadata_template='{key}: {value}', metadata_separator='\n', text='<a href="#challenges" ari